In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import KFold
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import np_utils

Using TensorFlow backend.


In [5]:
# def read_data(adjusted):
#     if adjusted:
#         data_file = pd.read_csv('MOVIES_ADJUSTED.csv', encoding='cp950')
#         return data_file
#     else:
#         data_file = pd.read_csv('MOVIES_WITHOUT_ADJUSTED.csv', encoding='cp950')
#         return data_file
def preprocessing(data_file):
    country = []
    genre = []
    date = []
    for i in range(data_length):
        t_country = str(data_file['COUNTRY'][i]).split(',')
        t_genre = str(data_file['IMDB_GENRE'][i]).split(',')
        t_date = str(data_file['DATE_TW'][i]).split('/')
        t_date = [ int(d) for d in t_date]
        country.append(t_country)
        genre.append(t_genre)
        date.append(t_date)
    mlb = MultiLabelBinarizer()
    country = mlb.fit_transform(country)
    genre = mlb.fit_transform(genre)
    date = np.array(date)
    runtime = np.array(data_file['IMDB_RUNTIME']).reshape(data_length, 1)
    dir_detail = np.array(data_file[['DIRECTOR_WINS', 'DIRECTOR_NOMINATIONS', 
                                     'DIRECTOR_RATINGS']])
    star_detail = np.array(data_file[['STAR_1_WINS', 'STAR_1_NOMINATIONS', 
                                      'STAR_1_RATINGS', 'STAR_2_WINS', 
                                      'STAR_2_NOMINATIONS', 'STAR_2_RATINGS', 
                                      'STAR_3_WINS', 'STAR_3_NOMINATIONS', 
                                      'STAR_3_RATINGS']])
    yahoo = np.array(data_file[['YAHOO_EVALUATION', 'YAHOO_VOTER']])
    PTT = np.array(data_file[['PTT_ARTICLE', 'PTT_PUSH', 'PTT_ARROW', 
                              'PTT_PULL', 'PTT_REPLY']])
    youtube = np.array(data_file[['YOUTUBE_VIEW', 'YOUTUBE_LIKE', 'YOUTUBE_DISLIKE']])
    x_train = np.concatenate((country, genre, runtime, dir_detail, star_detail, 
                          yahoo, PTT, youtube), axis=-1)
    rating = np.array(data_file['IMDB_RATING']).reshape(data_length, 1)
    return x_train, rating

In [14]:
# read training data
x_train, y_train = np.load('x_train.npy'), np.load('y_train.npy')
n_features = x_train.shape[1]

In [15]:
# Normalization
mean = np.mean(x_train, axis=0)
sigma = np.std(x_train, axis=0)
x_train = (x_train-mean)/(sigma + 1e-20)

In [16]:
# read testing data
x_test, y_test = np.load('x_test.npy'), np.load('y_test.npy')
x_test = (x_test-mean)/(sigma + 1e-20)

In [19]:
def rmse(y_true, y_pred):
    K.clip(y_pred, 1.0, 10.0)
    return K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
def create_model(dnn):
    model = Sequential()
    model.add(Dense(32, input_shape=(n_features,), activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    for units in dnn:
        model.add(Dense(units, activation='tanh'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
    model.add(Dense(1, activation='tanh'))
    model.add(Lambda(lambda x: x + K.constant(6.5558, dtype=K.floatx())))
    return model

In [29]:
kfold = KFold(n_splits=10, shuffle=True, random_state=5)
k = 0
test_score = []
for train_index, valid_index in kfold.split(x_train):
    k += 1
    X, X_V = x_train[train_index], x_train[valid_index]
    Y, Y_V = y_train[train_index], y_train[valid_index]
    
    # DNN
    dnn = [32, 32]
    model = create_model(dnn)
    callbacks = []
    callbacks.append(EarlyStopping(monitor='val_loss', patience=3))
    model.compile(loss='mse', optimizer='adam', metrics=[rmse])
    model.fit(X, Y, batch_size=32, validation_data=(X_V, Y_V), 
              epochs=100, callbacks=callbacks)
    test_score.append(model.predict(x_test))

Train on 159 samples, validate on 18 samples
Epoch 1/100
159/159 [==============================] - 2s - loss: 1.3382 - rmse: 1.0980 - val_loss: 0.7027 - val_rmse: 0.8018
Epoch 2/100
159/159 [==============================] - 0s - loss: 1.1135 - rmse: 1.0007 - val_loss: 0.4994 - val_rmse: 0.6987
Epoch 3/100
159/159 [==============================] - 0s - loss: 1.0152 - rmse: 0.9702 - val_loss: 0.4859 - val_rmse: 0.6916
Epoch 4/100
159/159 [==============================] - 0s - loss: 0.8217 - rmse: 0.8636 - val_loss: 0.4798 - val_rmse: 0.6848
Epoch 5/100
159/159 [==============================] - 0s - loss: 0.7069 - rmse: 0.7906 - val_loss: 0.5181 - val_rmse: 0.7087
Epoch 6/100
159/159 [==============================] - 0s - loss: 0.7520 - rmse: 0.8239 - val_loss: 0.5114 - val_rmse: 0.7043
Epoch 7/100
159/159 [==============================] - 0s - loss: 0.7074 - rmse: 0.7930 - val_loss: 0.5186 - val_rmse: 0.7087
Epoch 8/100
159/159 [==============================] - 0s - loss: 0.8750 

159/159 [==============================] - 0s - loss: 0.8135 - rmse: 0.8448 - val_loss: 0.2803 - val_rmse: 0.4872
Epoch 18/100
159/159 [==============================] - 0s - loss: 0.7623 - rmse: 0.8373 - val_loss: 0.2814 - val_rmse: 0.4885
Epoch 19/100
159/159 [==============================] - 0s - loss: 0.7060 - rmse: 0.7923 - val_loss: 0.2498 - val_rmse: 0.4575
Epoch 20/100
159/159 [==============================] - 0s - loss: 0.6673 - rmse: 0.7783 - val_loss: 0.2418 - val_rmse: 0.4562
Epoch 21/100
159/159 [==============================] - 0s - loss: 0.7310 - rmse: 0.7858 - val_loss: 0.2466 - val_rmse: 0.4571
Epoch 22/100
159/159 [==============================] - 0s - loss: 0.6987 - rmse: 0.7934 - val_loss: 0.2570 - val_rmse: 0.4652
Epoch 23/100
159/159 [==============================] - 0s - loss: 0.6485 - rmse: 0.7636 - val_loss: 0.2684 - val_rmse: 0.4811
Epoch 24/100
159/159 [==============================] - 0s - loss: 0.7019 - rmse: 0.7960 - val_loss: 0.2551 - val_rmse: 0.46

160/160 [==============================] - 0s - loss: 1.0863 - rmse: 0.9629 - val_loss: 0.6373 - val_rmse: 0.7620
Epoch 3/100
160/160 [==============================] - 0s - loss: 1.0834 - rmse: 0.9939 - val_loss: 0.6745 - val_rmse: 0.7698
Epoch 4/100
160/160 [==============================] - 0s - loss: 0.8900 - rmse: 0.8765 - val_loss: 0.7766 - val_rmse: 0.8186
Epoch 5/100
160/160 [==============================] - 0s - loss: 0.9687 - rmse: 0.9086 - val_loss: 0.7769 - val_rmse: 0.8046
Epoch 6/100
160/160 [==============================] - 0s - loss: 0.8483 - rmse: 0.8636 - val_loss: 0.6751 - val_rmse: 0.7554
Epoch 7/100
160/160 [==============================] - 0s - loss: 0.9114 - rmse: 0.8902 - val_loss: 0.7366 - val_rmse: 0.7993
Epoch 8/100
160/160 [==============================] - 0s - loss: 0.7917 - rmse: 0.8364 - val_loss: 0.7802 - val_rmse: 0.8162
Train on 160 samples, validate on 17 samples
Epoch 1/100
160/160 [==============================] - 2s - loss: 1.3266 - rmse: 1.10

In [30]:
# test accuracy
final_scores = np.zeros((len(x_test), 1))
for i in range(k):
    final_scores += test_score[i]
test_pred = final_scores / k
test_loss = [ np.abs(y_test[i] - test_pred[i])  for i in range(len(y_test))]
test_loss = np.sum(test_loss) / len(y_test)
print('test_acc: ' + str(test_loss))

test_acc: 0.757745652199
